![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Etapa 1: Instalação das dependências

In [1]:
!pip uninstall -y tensorflow #Comando necessário, pois o TensorFlow-gpu não desinstala a versão mais recente do Tensorflow, pode gerar conflitos.

Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0


In [2]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 450kB 47.5MB/s 
     |████████████████████████████████| 3.8MB 47.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8d90fd636c47232d7af9a8e3c940c32dc75e7456d2389eed5e0913c48686780d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2


In [3]:
!pip install tqdm

### Fazendo o download da base de dados de gatos e cachorros

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-07-14 12:46:05--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 108.177.126.128, 2a00:1450:4013:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  68.1MB/s    in 1.0s    

2020-07-14 12:46:06 (68.1 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [16]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

### Descompactando a base de dados de gatos e cachorros

In [17]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [18]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [19]:
zip_object.extractall("./")

In [20]:
zip_object.close()

### Configurando os caminhos (paths)

In [21]:
dataset_path_new = "./cats_and_dogs_filtered"

In [22]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [23]:
img_shape = (128, 128, 3)

In [24]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

In [25]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelando o modelo base

In [26]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [27]:
base_model.output

<tf.Tensor 'out_relu_1/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [28]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [29]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [37]:
dense_layer = tf.keras.layers.Dense(units=512, activation="relu", kernel_initializer="he_normal")(global_average_layer)
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(dense_layer)

### Definindo o modelo

In [38]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

### Compilando o modelo

In [40]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [34]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [35]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [41]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 11s 709ms/step - loss: 0.3784 - accuracy: 0.8360 - val_loss: 0.1842 - val_accuracy: 0.9400
Epoch 2/5
16/16 [==============================] - 9s 558ms/step - loss: 0.2134 - accuracy: 0.9280 - val_loss: 0.1776 - val_accuracy: 0.9260
Epoch 3/5
16/16 [==============================] - 9s 560ms/step - loss: 0.1657 - accuracy: 0.9440 - val_loss: 0.1747 - val_accuracy: 0.9270
Epoch 4/5
16/16 [==============================] - 9s 566ms/step - loss: 0.1396 - accuracy: 0.9560 - val_loss: 0.1250 - val_accuracy: 0.9510
Epoch 5/5
16/16 [==============================] - 9s 570ms/step - loss: 0.1243 - accuracy: 0.9560 - val_loss: 0.1624 - val_accuracy: 0.9340


### Avaliação do modelo de transferência de aprendizagem

In [42]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [43]:
valid_accuracy

0.934

## Fine tuning


Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 

### Descongelando algumas camadas do topo do modelo base

In [44]:
base_model.trainable = True
len(base_model.layers)

155

In [45]:
fine_tuning_at = 100

In [47]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

### Compilando o modelo para fine tuning

In [48]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [49]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 12s 725ms/step - loss: 0.0996 - accuracy: 0.9625 - val_loss: 0.0764 - val_accuracy: 0.9700
Epoch 2/5
16/16 [==============================] - 10s 602ms/step - loss: 0.0135 - accuracy: 0.9995 - val_loss: 0.0904 - val_accuracy: 0.9680
Epoch 3/5
16/16 [==============================] - 9s 587ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.1031 - val_accuracy: 0.9680
Epoch 4/5
16/16 [==============================] - 9s 594ms/step - loss: 0.0043 - accuracy: 0.9990 - val_loss: 0.1144 - val_accuracy: 0.9670
Epoch 5/5
16/16 [==============================] - 10s 594ms/step - loss: 9.9330e-04 - accuracy: 1.0000 - val_loss: 0.1226 - val_accuracy: 0.9660


### Avaliação do modelo com fine tuning

In [50]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [51]:
valid_accuracy

0.966